## PyRidy Example Notebook
The intent of this Jupyter Notebook to demonstrate and explain some of the capabilities of the PyRidy Library.

<div class="alert alert-block alert-info">
<b>Info:</b> PyRidy and the documentation are still in an early development phase. Therefore, features might change and the documentation is not yet complete
</div>

If pyridy is not installed in your environment, you can install it via:
```bash
    pip install pyridy
```
If pyridy is already installed, check whether you use the most recent version.
```shell
    pip install pyridy --upgrade
```

***

Documentation and API reference can be found here: <https://pyridy.readthedocs.io/en/latest/>

In [1]:
%load_ext autoreload
%autoreload 2

# Imports
import logging
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from scipy import signal

import pyridy

from pyridy.utils import AccelerationSeries, LinearAccelerationSeries, GyroSeries, GPSSeries # Every data series that should be used should be imported here
"""
The following TimeSeries classes currently exist:

    AccelerationSeries
    AccelerationUncalibratedSeries
    LinearAccelerationSeries
    MagnetometerSeries
    MagnetometerUncalibratedSeries
    NMEAMessageSeries
    GNSSClockMeasurementSeries
    GNSSMeasurementSeries
    OrientationSeries
    GyroSeries
    GyroUncalibratedSeries
    RotationSeries
    GPSSeries
    PressureSeries
    TemperatureSeries
    HumiditySeries
    LightSeries
    WzSeries
    SubjectiveComfortSeries
    
Whether these measurement were recorded, depends on the settings in the Ridy App and the device's capabilities.
"""

# logging.getLogger('pyridy').setLevel(logging.DEBUG)  # Uncomment this to see more information when pyridy functions are called

%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

### Loading measurement files
Within the folder two measurements files can be found that are going to be imported. The argument download_osm_region tries to download additional OSM which includes
railway track data, railway lines, position of level crossings and switches. Sometimes the Overpass API which is used to access the data can be overloaded leading to an error.

In [3]:
# Loading multiple files by creating a measurement campaign
path = "ridy_data"
#path = r"D://10_Daten/Ridy/Neu"

# campaign = pyridy.Campaign(folder=path, sync_method="ntp_time", download_osm_data=True, railway_types=["rail"], osm_recurse_type="<")

# Load only some timeseries
campaign = pyridy.Campaign(folder=path, sync_method="ntp_time", download_osm_data=False, railway_types=["rail"], osm_recurse_type="<", series=[GPSSeries])

0it [00:00, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

(2021-10-23T183518.175+0200d.sqlite) No ntp timestamp and datetime, falling back to device_time synchronization


  0%|          | 0/2 [00:00<?, ?it/s]

### Displaying the files GPS tracks on a map
The following code plots the GPS track onto a Map. If show_railway_elements is set to true, additional markers for switches and level crossings are being drawn onto the map. Depending on the size of the region creating the map can take several seconds.

In [7]:
m = campaign.create_map(show_railway_elements=False)
m

Map(center=[51.04085731506348, 6.595613241195679], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
campaign("2021-12-12T213211.405+0100.sqlite").measurements[AccelerationSeries].time[0]

### Plotting measurement series
The following code plots the velocity and linear acceleration of one vehicle over time. Due to the long measurement duration, it can take a few seconds until the plot appears.

In [ ]:
f = campaign[1] # Individual files can either be accessed over indices or by calling the campaign with the file name campaign("NAME_OF_THE_FILE")

fig, ax = plt.subplots(2, 1, sharex="row", figsize=(10, 5))

# Upper plot showing vehicle speed
ax[0].plot(f.measurements[GPSSeries].time, f.measurements[GPSSeries].speed*3.6)  # Note that individual data series can be accessed using their classes which have to be imported before
ax[0].grid()
ax[0].set_ylabel("Velocity [km/h]")
ax[0].set_xlabel("Time")
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# Lower plot showing vehicle vertical acceleration
ax[1].scatter(f.measurements[LinearAccelerationSeries].time, f.measurements[LinearAccelerationSeries].lin_acc_z, facecolors='none', edgecolors='r',s=3)
ax[1].grid()
ax[1].set_ylabel("Lin Acc_z [m/s^2]")
ax[1].set_xlabel("Time")

ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))


plt.suptitle(f.name)
plt.show()

### Get positions of railway track elements like switches, level crossings or signals
The downloaded OSM data contains information about the position of railway switches, level crossings or railway signals.

In [ ]:
switches = campaign.osm_region.get_switches()[:5] # Get the first five switches
switches

In [ ]:
switches[0].__dict__

In [ ]:
print(switches[0].lon, switches[0].lat) # Print e.g. the coordinates of the first switch

In [ ]:
signals = campaign.osm_region.get_signals() # Get railway signals
signals[20].__dict__ # Show the attributes of one exemplary signal

In [ ]:
level_crossings = campaign.osm_region.get_level_crossings() # Get level crossings
level_crossings[1].__dict__

### Get railway route information
The OSM data also contains information about that railway lines that use the downloaded tracks

In [ ]:
campaign.osm.railway_lines[:10] # Returns the first ten railway lines 

In [ ]:
campaign.osm.get_railway_line("RE1") # Search for a specific railway line

<div class="alert alert-block alert-info">
<b>Tip:</b> If you want to the view the attributes of an object (e.g. a switch or railway line), use ".__dict__"
</div>

It is also possible to retrieve the switches that are located on a specific line

In [ ]:
re1_aachen_köln = campaign.osm.get_railway_line("RE1")[1]
re1_switches = campaign.osm.get_switches_for_railway_line(re1_aachen_köln)
re1_switches[:5].lat

Plot retrieved switches onto the Map 

In [ ]:
from ipyleaflet import Icon, Marker

for sw in re1_switches:
    icon = Icon(icon_url='https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-black.png',
                shadow_url='https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
                icon_size=[25, 41],
                icon_anchor=[12, 41],
                popup_anchor=[1, -34],
                shadow_size=[41, 41])
    marker = Marker(location=(sw.lat, sw.lon), draggable=False, icon=icon)
    m.add(marker)
            
m